# Chapter 3. 제약 충족 문제
- 제약 충족 문제는 도메인이라는 범위에 속하는 값을 갖는 변수로 구성
- 변수 사이의 제약 조건을 충족하여 문제 해결

제약 충족 문제의 핵심 개념
- 제약 조건 변수(variables)
- 도메인
- 제약 조건(Constraint class) 
    - (충족 확인 메서드: satisfied())

해결 방법
- Prolog, Picat 같은 프로그래밍 언어는 제약 충족 문제를 해결할 수 있는 함수 제공
- 일반적으로는 백트래킹 검색과 이를 향상시키는 몇가지 휴리스틱을 통합하여 프레임워크 구축

이 장에서 살펴볼 것
- 재귀 백트래킹 검색을 사용한 프레임워크

In [ ]:
from typing import Generic, TypeVar, Dict, List, Optional
from abc import ABC, abstractmethod

V = TypeVar('V') # Variable type
D = TypeVar('D') # Domail type

# base class for Constraint
class Constraint(Generic[V, D], ABC):
    # variables
    def __init__(self, variables: List[V]) -> None:
        self.variables = variables

    @abstractmethod
    def satisfied(self, assignment: Dict[V, D]) -> bool:
        pass
